In [18]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time

In [19]:
## get problem parameters and geometry
problem = problems.problem3

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J
dH = lambda J: (1-1/1000) * phi(J) + np.log(1-phi(J)) + chi * phi(J)**2

gammafun = lambda lamb: -dH(lamb)/lamb # bonded
#gammafun = lambda lamb: -dH(lamb**3)*lamb

lambda_target = 1.49
gamma_target = gammafun(lambda_target)


gamma = Parameter(gamma_target*10)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F):
    
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*gamma* Trace(F.trans*F ) + H


def Gel_energy_EDP(F): ## |F|^2 + H => gamma F:Gradv + H'*J'
    # ddet(A(t))/dt = det(A(t))*trace(A^-1(t)*Grad (v))
    
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    invF = Inv(F)
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    edp = gamma * InnerProduct(F,dv) + H_prime * J * Trace(invF*dv)
    return edp


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = 0)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)
#F = Id(dim) + Grad(u)

## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(1)

uk = GridFunction(fesu)
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

phi = CF(0)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psih, psik = G_funs
    F = Id(3)+ grad(u)
    if form == "Functional":
        #LHS
        
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx)
        BF += delta * v[2]*dx
        BF += u[2]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
        #RHS
        BF += -(psik - psih)*v[2]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    elif form == "EDP":
        BF += alpha * Gel_energy_EDP(F).Compile() * dx
        BF += delta * v[2]*dx
        BF += u[2]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
        #RHS
        BF += -(psik - psih)*v[2]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    






In [20]:
max_PG_it = 5
max_iter_qnewton = 3
max_iter_newton = 3
newton_damp = 0.5



for k in range(1,max_PG_it):
    print("PG it:",k)
    alpha.Set(2**k)
    psik.vec.data = psih.vec
    print("Starting QN")
    for i in range(max_iter_qnewton):
        #incremental softening...?
        gammas = np.flip(np.linspace(gamma_target, gamma_target*10))

        G_funs = u,v,alpha,delta,psih,psik 
        BF = Assemble_BF(BF,G_funs,form)
        gfu = GridFunction(fes)
        gfu.vec[:] = 0
        res = gfu.vec.CreateVector()
        w = gfu.vec.CreateVector()
        #iterations of newton for the NL problem at each QN step
        print("Solving nonlinear subproblem")
        for load in gammas:
            gamma.Set(load)
            it = 0
            for iter in range(max_iter_newton):
                print("Gamma load": it)
                
                BF.Apply(gfu.vec,res)
                BF.AssembleLinearization(gfu.vec)
                inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                w.data = newton_damp * inv * res
                gfu.vec.data += -w
        psih.vec.data = psih.vec + gfu.components[1].vec



        #update gfu_u, psih



PG it: 1
Starting QN
Solving nonlinear subproblem

UMFPACK V5.7.9 (Oct 20, 2019): WARNING: matrix is singular



NgException: UmfpackInverse: Numeric factorization failed.

In [4]:
Draw(gfu.components[0])

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene